In [12]:
import warnings
from typing import Sequence, Tuple, Union

import torch
import torch.nn as nn

from monai.networks.blocks.convolutions import Convolution, ResidualUnit
from monai.networks.layers.factories import Act, Norm
from monai.networks.layers.simplelayers import SkipConnection
from monai.utils import alias, export

import matplotlib.pyplot as plt
import pandas as pd
from monai.losses import DiceLoss
from monai.metrics import DiceMetric, compute_meandice
import torch
import torch.nn as nn
import numpy as np
import os
import torch.optim as opt
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
import cv2
import h5py
from scipy.ndimage.interpolation import zoom
import torchvision.transforms as T
import random
from scipy.ndimage.interpolation import zoom
from torch_geometric.nn import SAGEConv
from torch.cuda.amp import GradScaler, autocast
from monai.networks.nets import UNet
from Graph_GNN import UNet_GNN_pass, UNet_GNN_linear
import SimpleITK as sitk

In [13]:
start = []
end = []
for i in range(625):
    if i >= 0 and i <= 24:
        if i == 0:
            start.append(0)
            end.append(1)
            start.append(0)
            end.append(50)
        elif i == 24:
            start.append(24)
            end.append(23)
            start.append(24)
            end.append(49)
        else:
            start.append(i)
            end.append(i-1)
            start.append(i)
            end.append(i+1)
            start.append(i)
            end.append(i+25)
    elif (i >= 600 and i <= 624):
        if i == 600:
            start.append(600)
            end.append(575)
            start.append(600)
            end.append(601)
        elif i == 624:
            start.append(600)
            end.append(575)
            start.append(600)
            end.append(601)
        else:
            start.append(i)
            end.append(i-1)
            start.append(i)
            end.append(i+1)
            start.append(i)
            end.append(i-25)
    elif i % 25 == 0:
        start.append(i)
        end.append(i+1)
        start.append(i)
        end.append(i+25)
        start.append(i)
        end.append(i-25)
    elif i % 25 == 24:
        start.append(i)
        end.append(i-1)
        start.append(i)
        end.append(i+25)
        start.append(i)
        end.append(i-25)     
    else:
        start.append(i)
        end.append(i-1)
        start.append(i)
        end.append(i+25)
        start.append(i)
        end.append(i-25)  
        start.append(i)
        end.append(i+1)
edges = torch.tensor([start, end], dtype=torch.long)
    
def one_hot(labels: torch.Tensor, num_classes: int, dtype: torch.dtype = torch.float, dim: int = 1) -> torch.Tensor:
    # if `dim` is bigger, add singleton dim at the end
    if labels.ndim < dim + 1:
        shape = list(labels.shape) + [1] * (dim + 1 - len(labels.shape))
        labels = torch.reshape(labels, shape)

    sh = list(labels.shape)

    if sh[dim] != 1:
        raise AssertionError("labels should have a channel with length equal to one.")

    sh[dim] = num_classes

    o = torch.zeros(size=sh, dtype=dtype, device=labels.device)
    labels = o.scatter_(dim=dim, index=labels.long(), value=1)

    return labels

In [14]:
base = './'
data_path = [base + 'val/' + x for x in os.listdir(base + 'val/')]
names = [x.split('/')[-1].split('_')[0] for x in data_path]
counts = pd.Series(names).value_counts().to_dict()
templete =  './val/{x}_{y}.h5'

In [18]:
def save_one_case(name, counts, model, device, unet=False):
    count = counts[name]
    real = []
    out = []
    inp = []
    for i in range(1, count-2):
        paths = []
        for j in range(i-1, i+2):
            paths.append(templete.format(x=name, y=j))
        xs = []
        f = h5py.File(paths[1], 'r')
        y = f['gt'][:]
        real.append(y)
        inp.append(f['ct'][:])
        f.close()
        for path in paths:
            f = h5py.File(path, 'r')
            x = f['ct'][:].astype(np.float)
            x = torch.from_numpy(x)
            x = x.float().view(1, 400, 400)
            xs.append(x)  
            f.close()
        x = torch.cat(xs, dim=0).view(1, 3, 400, 400)
        x = x.to(device)
        if unet:
            outputs = model(x)
        else:
            outputs = model(x, device, edges)
        outputs = torch.nn.functional.softmax(outputs, 1)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.view(400, 400).detach().cpu().numpy()
        out.append(predicted)
        
    out = np.array(out)
    real = np.array(real)
    inp = np.array(inp)
    
    out = np.uint8(out)
    real = np.uint8(real)
    
    print(out.shape, inp.shape, real.shape)
    
    
    return inp, real, out

In [19]:
def test_one_model(model, device, unet, model_name):
    model.to(device)
    model.eval()
    for name in tqdm(counts.keys()):
        inp, real, out = save_one_case(name, counts, model, device, unet=unet)
        os.makedirs('./3dtestoutput/{x}/'.format(x = name), exist_ok=True)
        if model_name=='unet':
            img_nii = sitk.GetImageFromArray(inp)
            loc =  './3dtestoutput/{x}/'.format(x = name) + 'ct.nii.gz'
            sitk.WriteImage(img_nii, loc)
            
            img_nii = sitk.GetImageFromArray(real)
            loc =  './3dtestoutput/{x}/'.format(x = name) + 'grount_truth.nii.gz'
            sitk.WriteImage(img_nii, loc) 
            
            img_nii = sitk.GetImageFromArray(out)
            loc =  './3dtestoutput/{x}/'.format(x = name) + 'unet.nii.gz'
            sitk.WriteImage(img_nii, loc)
            
        elif model_name=='linear':
            img_nii = sitk.GetImageFromArray(out)
            loc =  './3dtestoutput/{x}/'.format(x = name) + 'gnn_unet_linear.nii.gz'
            sitk.WriteImage(img_nii, loc)    
        
        else:
            img_nii = sitk.GetImageFromArray(out)
            loc =  './3dtestoutput/{x}/'.format(x = name) + 'gnn_unet_pass.nii.gz'
            sitk.WriteImage(img_nii, loc)

In [20]:
model_unet = UNet(
    dimensions=2,
    in_channels=3,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
)
model_unet.load_state_dict(torch.load('./model_weights/best_unet_weight.pt'))
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
test_one_model(model_unet, device, True, 'unet')
del model_unet

KeyboardInterrupt: 

In [ ]:
model_gnn_pass = UNet_GNN_pass(
    dimensions=2,
    in_channels=3,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
)
model_gnn_pass.load_state_dict(torch.load('./model_weights/best_sage_conv_weight.pt'))
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
test_one_model(model_gnn_pass, device, False, 'pass')
del model_gnn_pass

In [ ]:
model_gnn_linear = UNet_GNN_linear(
    dimensions=2,
    in_channels=3,
    out_channels=2,
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
)
model_gnn_linear.load_state_dict(torch.load('./model_weights/best_sage_conv_linear_weight.pt'))
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
get_stat(model_gnn_linear, device, 'linear')
del model_gnn_linear